In [1]:
!nvidia-smi

Tue Dec 30 08:42:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
%%writefile seq_sort.cu

#include <iostream>                     // Для ввода/вывода (cout, endl)
#include <vector>                       // Для использования динамических массивов vector
#include <algorithm>                    // Для функций swap и min
#include <chrono>                       // Для измерения времени выполнения
#include <random>                       // Для генерации случайных чисел
#include <cuda_runtime.h>               // Для работы с CUDA API

using namespace std;                    // Чтобы не писать std::

// CPU Сортировки

// MergeSort CPU: функция слияния двух подмассивов
void merge(vector<int>& arr, int left, int mid, int right){
    int n1 = mid-left+1;                 // Размер левой части массива
    int n2 = right-mid;                  // Размер правой части массива
    vector<int> L(n1), R(n2);            // Временные массивы для слияния
    for(int i=0;i<n1;i++) L[i] = arr[left+i]; // Копируем элементы левой части
    for(int i=0;i<n2;i++) R[i] = arr[mid+1+i]; // Копируем элементы правой части

    int i=0,j=0,k=left;                  // Индексы для левой части, правой части и итогового массива
    while(i<n1 && j<n2){                  // Пока есть элементы в обеих частях
        if(L[i]<=R[j]) arr[k++] = L[i++]; // Копируем меньший элемент в итоговый массив
        else arr[k++] = R[j++];
    }
    while(i<n1) arr[k++] = L[i++];       // Копируем остатки левой части, если есть
    while(j<n2) arr[k++] = R[j++];       // Копируем остатки правой части, если есть
}

// Рекурсивная функция MergeSort
void mergeSort(vector<int>& arr,int left,int right){
    if(left<right){
        int mid = left + (right-left)/2; // Находим середину массива
        mergeSort(arr,left,mid);         // Рекурсивно сортируем левую половину
        mergeSort(arr,mid+1,right);      // Рекурсивно сортируем правую половину
        merge(arr,left,mid,right);       // Сливаем две отсортированные половины
    }
}

// QuickSort CPU: функция разделения
int partition(vector<int>& arr,int low,int high){
    int pivot = arr[high];               // Опорный элемент (последний элемент)
    int i = low-1;                        // Индекс меньшего элемента
    for(int j=low;j<high;j++){
        if(arr[j]<=pivot){ i++; swap(arr[i],arr[j]); } // Перемещаем элементы меньше pivot
    }
    swap(arr[i+1],arr[high]);            // Перемещаем pivot на правильное место
    return i+1;                           // Возвращаем индекс pivot
}

// QuickSort рекурсивно
void quickSort(vector<int>& arr,int low,int high){
    if(low<high){
        int pi = partition(arr,low,high); // Разделение массива
        quickSort(arr,low,pi-1);          // Сортируем левую часть
        quickSort(arr,pi+1,high);         // Сортируем правую часть
    }
}

// HeapSort CPU: функция для корректировки кучи
void heapify(vector<int>& arr,int n,int i){
    int largest = i;                     // Предполагаем, что корень - наибольший
    int l = 2*i+1;                       // Левый потомок
    int r = 2*i+2;                       // Правый потомок
    if(l<n && arr[l]>arr[largest]) largest = l; // Проверяем левый потомок
    if(r<n && arr[r]>arr[largest]) largest = r; // Проверяем правый потомок
    if(largest!=i){ swap(arr[i],arr[largest]); heapify(arr,n,largest); } // Меняем местами и рекурсивно проверяем поддерево
}

// HeapSort CPU
void heapSort(vector<int>& arr){
    int n = arr.size();
    for(int i=n/2-1;i>=0;i--) heapify(arr,n,i); // Построение кучи
    for(int i=n-1;i>=0;i--){                    // Извлечение элементов из кучи
        swap(arr[0],arr[i]);                    // Перемещаем корень в конец массива
        heapify(arr,i,0);                       // Восстанавливаем кучу
    }
}

// GPU MergeSort

// Kernel для параллельного слияния подмассивов
__global__ void gpuMergeKernel(int* arr,int* temp,int width,int size){
    int tid = blockIdx.x*blockDim.x + threadIdx.x; // Индекс потока
    int start = tid*width*2;                        // Начало подмассива
    if(start>=size) return;                         // Выход, если за пределами массива

    int mid = min(start+width-1,size-1);           // Середина подмассива
    int end = min(start+2*width-1,size-1);         // Конец подмассива

    int i=start,j=mid+1,k=start;                   // Индексы для слияния
    while(i<=mid && j<=end){
        if(arr[i]<=arr[j]) temp[k++] = arr[i++];   // Копируем меньший элемент
        else temp[k++] = arr[j++];
    }
    while(i<=mid) temp[k++] = arr[i++];            // Остатки левой части
    while(j<=end) temp[k++] = arr[j++];            // Остатки правой части
}

// GPU MergeSort wrapper
void gpuMergeSort(int* d_arr,int size){
    int* d_temp;
    cudaMalloc(&d_temp,size*sizeof(int));          // Выделяем временный массив на GPU
    int width=1;
    while(width<size){                              // Многоступенчатое слияние
        int numThreads = (size + 2*width -1)/(2*width);     // Количество потоков
        int blockSize = 256;                                 // Размер блока
        int numBlocks = (numThreads + blockSize -1)/blockSize; // Количество блоков
        gpuMergeKernel<<<numBlocks,blockSize>>>(d_arr,d_temp,width,size); // Запуск kernel
        cudaDeviceSynchronize();                               // Синхронизация
        int* tmp = d_arr; d_arr = d_temp; d_temp = tmp;       // Меняем массивы местами
        width*=2;                                             // Увеличиваем ширину подмассива
    }
    cudaFree(d_temp);                                         // Освобождаем временный массив
}

// GPU QuickSort
__global__ void gpuQuickSortKernel(int* arr,int size){
    if(threadIdx.x!=0) return;                          // Только один поток для демонстрации
    int low=0,high=size-1;
    int stack[1024]; int top=-1;
    stack[++top]=low; stack[++top]=high;
    while(top>=0){
        high=stack[top--]; low=stack[top--];
        int pivot = arr[high]; int i=low-1;
        for(int j=low;j<high;j++){
            if(arr[j]<=pivot){ i++; int tmp=arr[i]; arr[i]=arr[j]; arr[j]=tmp; }
        }
        int tmp=arr[i+1]; arr[i+1]=arr[high]; arr[high]=tmp;
        int pi=i+1;
        if(pi-1>low){ stack[++top]=low; stack[++top]=pi-1; }
        if(pi+1<high){ stack[++top]=pi+1; stack[++top]=high; }
    }
}

// GPU HeapSort
__global__ void gpuHeapSortKernel(int* arr,int size){
    if(threadIdx.x!=0) return; // Один поток для демонстрации
    int n=size;
    for(int i=n/2-1;i>=0;i--){
        int largest=i,l=2*i+1,r=2*i+2;
        if(l<n && arr[l]>arr[largest]) largest=l;
        if(r<n && arr[r]>arr[largest]) largest=r;
        if(largest!=i){ int tmp=arr[i]; arr[i]=arr[largest]; arr[largest]=tmp; }
    }
    for(int i=n-1;i>=0;i--){ int tmp=arr[0]; arr[0]=arr[i]; arr[i]=tmp; }
}

//  Генерация массива
vector<int> generateRandomArray(int size){
    vector<int> arr(size);
    random_device rd;                              // Источник случайных чисел
    mt19937 gen(rd());                              // Генератор случайных чисел
    uniform_int_distribution<> dis(1,1000000);     // Диапазон чисел
    for(int i=0;i<size;i++) arr[i]=dis(gen);       // Заполнение массива случайными числами
    return arr;
}

// Сравнение производительности
void compareSorts(int size){
    cout<<"Размер массива N = "<<size <<endl;
    vector<int> arr = generateRandomArray(size);      // Исходный массив
    vector<int> arrCPU;                               // Массив для CPU
    int* d_arr;
    cudaMalloc(&d_arr,size*sizeof(int));
    cudaMemcpy(d_arr,arr.data(),size*sizeof(int),cudaMemcpyHostToDevice);

    // MergeSort CPU
    arrCPU=arr;
    auto start=chrono::high_resolution_clock::now();
    mergeSort(arrCPU,0,arrCPU.size()-1);
    auto end=chrono::high_resolution_clock::now();
    double mergeCPU = chrono::duration<double>(end-start).count();

    // MergeSort GPU
    start=chrono::high_resolution_clock::now();
    gpuMergeSort(d_arr,size);
    cudaDeviceSynchronize();
    end=chrono::high_resolution_clock::now();
    double mergeGPU = chrono::duration<double>(end-start).count();

    // QuickSort CPU
    arrCPU=arr;
    start=chrono::high_resolution_clock::now();
    quickSort(arrCPU,0,arrCPU.size()-1);
    end=chrono::high_resolution_clock::now();
    double quickCPU = chrono::duration<double>(end-start).count();

    // QuickSort GPU (демо)
    start=chrono::high_resolution_clock::now();
    gpuQuickSortKernel<<<1,1>>>(d_arr,size);
    cudaDeviceSynchronize();
    end=chrono::high_resolution_clock::now();
    double quickGPU = chrono::duration<double>(end-start).count();

    // HeapSort CPU
    arrCPU=arr;
    start=chrono::high_resolution_clock::now();
    heapSort(arrCPU);
    end=chrono::high_resolution_clock::now();
    double heapCPU = chrono::duration<double>(end-start).count();

    // HeapSort GPU (демо)
    start=chrono::high_resolution_clock::now();
    gpuHeapSortKernel<<<1,1>>>(d_arr,size);
    cudaDeviceSynchronize();
    end=chrono::high_resolution_clock::now();
    double heapGPU = chrono::duration<double>(end-start).count();

    // Вывод всех замеров времени
    cout<<"mergeCPU: "<<mergeCPU<<"s,\n mergeGPU: "<<mergeGPU
        <<"s, \n quickCPU: "<<quickCPU<<"s, \n quickGPU: "<<quickGPU
        <<"s,\n heapCPU: "<<heapCPU<<"s,\n heapGPU: "<<heapGPU<<"s"<<endl;

    cudaFree(d_arr);                                 // Освобождение памяти GPU

}

// main
int main(){
    compareSorts(10000);      // Массив 10 000 элементов
    compareSorts(100000);     // Массив 100 000 элементов
    compareSorts(1000000);    // Массив 1 000 000 элементов
    return 0;
}




Overwriting seq_sort.cu


In [20]:
# Компиляция и запуск в Colab
!nvcc seq_sort.cu -o seq_sort
!./seq_sort

Размер массива N = 10000
mergeCPU: 0.00956156s,
 mergeGPU: 0.0122674s, 
 quickCPU: 0.00310908s, 
 quickGPU: 1.0115e-05s,
 heapCPU: 0.00587938s,
 heapGPU: 1.126e-05s
Размер массива N = 100000
mergeCPU: 0.102561s,
 mergeGPU: 5.3893e-05s, 
 quickCPU: 0.0315494s, 
 quickGPU: 1.562e-05s,
 heapCPU: 0.056312s,
 heapGPU: 9.132e-06s
Размер массива N = 1000000
mergeCPU: 0.820595s,
 mergeGPU: 0.000349726s, 
 quickCPU: 0.386261s, 
 quickGPU: 2.6353e-05s,
 heapCPU: 0.747908s,
 heapGPU: 2.4899e-05s
